# Gourmet Meals Business -- SQL Project (Part 2.5 - Cleaning Data)

Author: **Ethan Moody**

Date: **October 2022**

### Business Case

Assume you are a data engineer working closely with the data science team at Agile Gourmet Meals (AGM).

AGM executives are considering adding a delivery option, with the hopes of increasing sales, growing the customer base, and increasing profitability.   

Management decided to do a proof of concept (POC) in the form of a three month trial run using one delivery service at the Berkeley store. They have called upon the data science team to help with this effort. In turn, the data science team has asked for your help in the data engineering aspects of the POC.

Management chose Peak Deliveries primarily because it's a newer operation with a model that takes a percentage cut of the product pricing instead of charging customers a delivery fee. Peak's cut is 18%. So, for each $12 meal, that equates to approximately $2.16. Customers may tip the delivery driver if they wish. AGM is not given any visibility into customer tips. (Peak is protecting its data on good tippers.) Peak has an outstanding reputation for great, fast, and efficient deliveries, with excellent customer service. Peak will only deliver to zip codes within a 5 mile radius of the store.

Integration with any third party sales channel always comes with its challenges. For large companies, like McDonalds, the delivery companies are willing to integrate and modify their computer systems as needed to get the contract. For small companies, like AGM, one of your only options is to use Peak's API to send and receive data. However, that would require you to write a lot of code, which management does not want to spend money on until the POC has proven successful. As an alternative, Peak can provide you with a JSON file at the end of each day with detailed sales information for that day. Management has decided to go with the daily JSON option for now for the POC. 

For products, AGM will enter products into Peak's system. Peak will assign an ID in their system to the product. You will need to create a mapping table to map Peak's IDs to AGM's IDs. In AGM's case, all products cost $12 and are tax exempt. AGM will mark them as exempt from sales tax.

Regarding the customer list, AGM does not want to give out their full customer list to third parties.  Customers will have to sign up with Peak, either using the website, the app, or by telephone.  AGM executives anticipate and understand that the trade off to not giving them the customer list is that you will probably have to validate and/or cleanse the customer data. Peak will assign their customer ID to each customer.

In this POC, you will focus on only 1 store: the Berkeley store. Peak will create a pickup location for the store and assign their own location ID to it. Even though all data will have the same store for now, you still want to receive it and process it so you can help leadership plan for possible future expansion to other stores and/or pickup locations.

Assume today is October 4, 2020. The first day of sales was October 3, 2020. The JSON file came in very early this morning. As a data engineer, you need to get started with parsing, staging, validating, etc. the file as soon as possible.  

The executives are anxious to understand how good the data is, if you will be able to continue withholding the customer data from Peak, and to get some preliminary analytics. Even though it's just one day's worth of data, the executives want as much information as soon as they can get it (which is very typical).

The data science team has met with you, and together you came up with a plan to get the data loaded and validated, explore the customer data, and perform some preliminary analytics. The data science team has been requested to give the executives an assessment of the customer data and whether or not they should continue to withhold customer data from Peak. Since you are going to be the first one to have an extensive look at the data, the data science team wants and values your opinion on the customer data.

# Included Modules and Packages

In [1]:
import csv
import json
import math
import numpy as np
import pandas as pd
import psycopg2

# Additional Setup Code

In [2]:
# Function to run a select query and return rows in a pandas dataframe
# Note: pandas formats all numeric values from postgres as float

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "Function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # Fix any float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
# Set up connection to postgres
# Note: All connection inputs below have been removed for protection
connection = psycopg2.connect(
    user = "",
    password = "",
    host = "",
    port = "",
    database = ""
)

In [4]:
cursor = connection.cursor()

# 2.5.1 Validate the city, state, and zip for stage_1_peak_customers against the zip_codes table

In [5]:
# Query validates that all city, state, and zip entries in the staging table stage_1_peak_customers match records in the zip_codes table
# Note: Query should return 0 rows

rollback_before_flag = True
rollback_after_flag = True

query = """

select
  t1_s_pcustomers.stage_id

from stage_1_peak_customers as t1_s_pcustomers

join zip_codes as t2_zip_codes
on t1_s_pcustomers.zip = t2_zip_codes.zip

where t1_s_pcustomers.city <> t2_zip_codes.city
and t1_s_pcustomers.state <> t2_zip_codes.state
and t1_s_pcustomers.zip <> t2_zip_codes.zip

order by
  t1_s_pcustomers.stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# 2.5.2 Find all customer records in stage_1_peak_customers where any of first_name, last_name, and/or street do not match a customer in the customers table

In [6]:
# Query identifies all records in the staging table stage_1_peak_customers whose first name, last name, and/or street do not match a customer in the customers table

rollback_before_flag = True
rollback_after_flag = True

query = """

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.first_name = t2_customers.first_name

where t2_customers.first_name is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.last_name = t2_customers.last_name

where t2_customers.last_name is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.street = t2_customers.street

where t2_customers.street is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.first_name = t2_customers.first_name
and t1_s_pcustomers.last_name = t2_customers.last_name

where t2_customers.first_name is null
and t2_customers.last_name is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.first_name = t2_customers.first_name
and t1_s_pcustomers.street = t2_customers.street

where t2_customers.first_name is null
and t2_customers.street is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.last_name = t2_customers.last_name
and t1_s_pcustomers.street = t2_customers.street

where t2_customers.last_name is null
and t2_customers.street is null
)

union

(
select
  t1_s_pcustomers.stage_id
, t1_s_pcustomers.sale_id::numeric
, t1_s_pcustomers.customer_id::numeric
, t1_s_pcustomers.first_name
, t1_s_pcustomers.last_name
, t1_s_pcustomers.street
, t1_s_pcustomers.city
, t1_s_pcustomers.state
, t1_s_pcustomers.zip

from stage_1_peak_customers as t1_s_pcustomers

left outer join customers as t2_customers
on t1_s_pcustomers.first_name = t2_customers.first_name
and t1_s_pcustomers.last_name = t2_customers.last_name
and t1_s_pcustomers.street = t2_customers.street

where t2_customers.first_name is null
and t2_customers.last_name is null
and t2_customers.street is null
)

order by
  stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,customer_id,first_name,last_name,street,city,state,zip
0,10,5763728768,3729016,Hyrum,Knuckles,86668 Spenser Terrace,Oakland,CA,94618
1,20,5763728877,3728936,Roseann,Coyish,11707 American Ash Ter,Orinda,CA,94563
2,24,5763728428,3729287,Hali,Ducker,8 Orion Pass,El Cerrito,CA,94530
3,26,5763728393,3728674,Melantha,Golborn,6140 North Field Alley,Orinda,CA,94563
4,36,5763729212,3729191,Eleni,Jansen,66 Bartelt Hill,Oakland,CA,94607
5,40,5763729129,3728856,Clyve,Humonds,22 Brent Wood Hill,Berkeley,CA,94709
6,51,5763728864,3729178,Rutledge,Hellwing,606 Gulf Plz,El Cerrito,CA,94530
7,60,5763729313,3728402,Kalli,Kemel,18373 Golf View Pass,Berkeley,CA,94702
8,72,5763728980,3729213,Honina,Philson,28 Clarendon Plaza,Berkeley,CA,94702
9,73,5763728921,3729194,Nicky,Haley,88424 Warrior Lane,Oakland,CA,94602


# 2.5.3 Find the percentage of Peak's customer records that do not match to AGM's customers table

In [7]:
# Query shows the percentage of Peak's customer records that do not match to AGM's customers table

rollback_before_flag = True
rollback_after_flag = True

query = """

with
  
  t1v_total_peak_customer_count as
  (
  select
    count(t1_s_pcustomers.stage_id) as customer_count

  from stage_1_peak_customers as t1_s_pcustomers
  ),
  
  t2v_total_matching_customer_count as
  (
  select
    count(sqa_total_matching_customers.stage_id) as matching_customer_count

  from
    (
      (
      select
        t1_s_pcustomers.stage_id
      , t1_s_pcustomers.sale_id::numeric
      , t1_s_pcustomers.customer_id::numeric
      , t1_s_pcustomers.first_name
      , t1_s_pcustomers.last_name
      , t1_s_pcustomers.street
      , t1_s_pcustomers.city
      , t1_s_pcustomers.state
      , t1_s_pcustomers.zip

      from stage_1_peak_customers as t1_s_pcustomers

      join customers as t2_customers
      on t1_s_pcustomers.first_name = t2_customers.first_name
      and t1_s_pcustomers.last_name = t2_customers.last_name
      and t1_s_pcustomers.street = t2_customers.street

      order by
        stage_id
      )
    ) as sqa_total_matching_customers
  )

select
  t1v_total_peak_customer_count.customer_count as total_peak_customers
, t2v_total_matching_customer_count.matching_customer_count as total_matching_customers
, t1v_total_peak_customer_count.customer_count - t2v_total_matching_customer_count.matching_customer_count as total_not_matching_customers
, round((sum(t1v_total_peak_customer_count.customer_count - t2v_total_matching_customer_count.matching_customer_count) / sum(t1v_total_peak_customer_count.customer_count)) * 100, 2) as percent_not_matching_customers

from
  t1v_total_peak_customer_count
, t2v_total_matching_customer_count

group by
  t1v_total_peak_customer_count.customer_count
, t2v_total_matching_customer_count.matching_customer_count

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,total_peak_customers,total_matching_customers,total_not_matching_customers,percent_not_matching_customers
0,97,84,13,13.4
